### Importing libraries

In [1]:
from pyspark.sql import SparkSession
import pandas as pd
import os
import geopandas as gpd
import folium

## Understanding data

In Newyork City, all taxi vehicles are managed by TLC (Taxi and Limousine Commission). The TLC Trip Record data to public for research and study purposes. The dataset in this project is of Uber, Lyft and Via which are one of the leading taxi service providers in New York.

The entire dataset include 5 kinds of data:
- **Dataset 1:**  *taxi_zone_lookup.csv* : stores taxi zones zip code and other relevant information.
- **Dataset 2:**  *nyc 2021-01-01 to 2021-12-31.csv* : Daily weather data of New york for 2021
- **Dataset 3:**  *taxi-zones* : folder contains the geospatial data of NYC taxi zones
- **Dataset 4:**  *fhvhv_tripdata_2021-01.parquet to fhvhv_tripdata_2021-12.parquet* : 12 files. Containing data of taxi trips of each month.
- **Dataset 5:**  *public_holidays.csv* : Contains list of public holidays in 2021
- **Dataset 6:** *weather.csv* : Hourly weather data of 2021

### Dataset 1: taxi_zone_lookup.csv

In [2]:
taxi_zone_lookup = pd.read_csv('dataset/taxi_zone_lookup.csv')

In [3]:
taxi_zone_lookup.sample(5)

,LocationID,Borough,Zone,service_zone
111,112,Brooklyn,Greenpoint,Boro Zone
123,124,Queens,Howard Beach,Boro Zone
46,47,Bronx,Claremont/Bathgate,Boro Zone
224,225,Brooklyn,Stuyvesant Heights,Boro Zone
124,125,Manhattan,Hudson Sq,Yellow Zone


In [4]:
taxi_zone_lookup[taxi_zone_lookup['LocationID']==167]

,LocationID,Borough,Zone,service_zone
166,167,Bronx,Morrisania/Melrose,Boro Zone


In [5]:
taxi_zone_lookup.shape

(265, 4)

In [6]:
taxi_zone_lookup.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 265 entries, 0 to 264
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   LocationID    265 non-null    int64 
 1   Borough       265 non-null    object
 2   Zone          264 non-null    object
 3   service_zone  263 non-null    object
dtypes: int64(1), object(3)
memory usage: 8.4+ KB


In [7]:
taxi_zone_lookup['Borough'].value_counts()

Borough
Queens           69
Manhattan        69
Brooklyn         61
Bronx            43
Staten Island    20
Unknown           2
EWR               1
Name: count, dtype: int64

In [8]:
taxi_zone_lookup['Zone'].value_counts()

Zone
Governor's Island/Ellis Island/Liberty Island    3
Corona                                           2
Newark Airport                                   1
Ocean Hill                                       1
Parkchester                                      1
                                                ..
Fordham South                                    1
Forest Hills                                     1
Forest Park/Highland Park                        1
Fort Greene                                      1
NV                                               1
Name: count, Length: 261, dtype: int64

In [9]:
taxi_zone_lookup['service_zone'].value_counts()

service_zone
Boro Zone      205
Yellow Zone     55
Airports         2
EWR              1
Name: count, dtype: int64

In [10]:
for zone in taxi_zone_lookup['service_zone'].unique():
    print(zone)
    print(taxi_zone_lookup[taxi_zone_lookup['service_zone']==zone]['Borough'].value_counts())
    print()

EWR
Borough
EWR    1
Name: count, dtype: int64

Boro Zone
Borough
Queens           67
Brooklyn         61
Bronx            43
Staten Island    20
Manhattan        14
Name: count, dtype: int64

Yellow Zone
Borough
Manhattan    55
Name: count, dtype: int64

Airports
Borough
Queens    2
Name: count, dtype: int64

nan
Series([], Name: count, dtype: int64)



In [11]:
taxi_zone_lookup[taxi_zone_lookup['service_zone'].isnull()]

,LocationID,Borough,Zone,service_zone
263,264,Unknown,NV,NaN
264,265,Unknown,NaN,NaN


### Dataset 2: nyc weather

In [12]:
weather = pd.read_csv('dataset/nyc_2021_weather.csv')

In [13]:
weather.sample(5)

,name,address,resolvedAddress,datetime,temp,feelslike,dew,humidity,precip,precipprob,...,snow,snowdepth,windgust,windspeed,winddir,sealevelpressure,cloudcover,visibility,uvindex,severerisk
317,nyc,nyc,"New York, NY, United States",2021-11-14,6.5,4.4,-0.5,61.3,0.32,100,...,0.0,0.0,42.5,23.8,240.1,1012.0,42.7,15.9,4,NaN
141,nyc,nyc,"New York, NY, United States",2021-05-22,25.3,24.9,12.7,47.2,0.00,0,...,0.0,0.0,33.5,21.2,251.1,1021.5,43.8,16.0,8,NaN
130,nyc,nyc,"New York, NY, United States",2021-05-11,14.3,14.3,0.9,42.8,0.00,0,...,0.0,0.0,50.5,23.5,300.8,1016.5,50.3,16.0,9,NaN
28,nyc,nyc,"New York, NY, United States",2021-01-29,-6.2,-14.1,-17.2,41.7,0.00,0,...,0.0,1.4,62.2,43.3,303.8,1019.8,18.1,16.0,6,NaN
5,nyc,nyc,"New York, NY, United States",2021-01-06,3.0,-0.6,-3.3,63.7,0.00,0,...,0.0,0.0,44.7,28.8,300.9,1015.0,46.6,16.0,5,NaN


In [14]:
weather.shape

(365, 21)

In [15]:
weather.columns

Index(['name', 'address', 'resolvedAddress', 'datetime', 'temp', 'feelslike',
       'dew', 'humidity', 'precip', 'precipprob', 'preciptype', 'snow',
       'snowdepth', 'windgust', 'windspeed', 'winddir', 'sealevelpressure',
       'cloudcover', 'visibility', 'uvindex', 'severerisk'],
      dtype='object')

In [16]:
weather[['name','address','resolvedAddress']].value_counts()

name  address  resolvedAddress            
nyc   nyc      New York, NY, United States    365
Name: count, dtype: int64

In [17]:
# Have only one value
drop_columns = ['name','address','resolvedAddress']

In [18]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              365 non-null    object 
 1   address           365 non-null    object 
 2   resolvedAddress   365 non-null    object 
 3   datetime          365 non-null    object 
 4   temp              365 non-null    float64
 5   feelslike         365 non-null    float64
 6   dew               365 non-null    float64
 7   humidity          365 non-null    float64
 8   precip            365 non-null    float64
 9   precipprob        365 non-null    int64  
 10  preciptype        152 non-null    object 
 11  snow              365 non-null    float64
 12  snowdepth         365 non-null    float64
 13  windgust          292 non-null    float64
 14  windspeed         365 non-null    float64
 15  winddir           365 non-null    float64
 16  sealevelpressure  365 non-null    float64
 1

In [19]:
# Columns with null values = [preciptype, windgust, severerisk]

In [20]:
drop_columns.append('severerisk')

**Filling missing values**

In [21]:
# Preciptype

In [22]:
weather['preciptype'].value_counts()

preciptype
rain         132
rain,snow     17
snow           3
Name: count, dtype: int64

In [23]:
weather[weather['preciptype']=='rain'][['precip','precipprob']].describe()

,precip,precipprob
count,132.000000,132.0
mean,9.790530,100.0
std,19.339919,0.0
min,0.040000,100.0
25%,0.802500,100.0
50%,3.665000,100.0
75%,11.725000,100.0
max,165.380000,100.0


In [24]:
weather[weather['preciptype']=='rain,snow'][['precip','precipprob']].describe()

,precip,precipprob
count,17.000000,17.0
mean,6.221176,100.0
std,10.446702,0.0
min,0.070000,100.0
25%,0.480000,100.0
50%,2.660000,100.0
75%,5.090000,100.0
max,39.960000,100.0


In [25]:
weather[weather['preciptype']=='snow'][['precip','precipprob']].describe()

,precip,precipprob
count,3.000000,3.0
mean,4.356667,100.0
std,5.677960,0.0
min,0.270000,100.0
25%,1.115000,100.0
50%,1.960000,100.0
75%,6.400000,100.0
max,10.840000,100.0


In [26]:
weather[(weather['preciptype'].isnull()) & (weather['precipprob']==100)]

,name,address,resolvedAddress,datetime,temp,feelslike,dew,humidity,precip,precipprob,...,snow,snowdepth,windgust,windspeed,winddir,sealevelpressure,cloudcover,visibility,uvindex,severerisk


In [27]:
weather[weather['preciptype'].isnull()][['precip','precipprob']].describe()

,precip,precipprob
count,213.0,213.0
mean,0.0,0.0
std,0.0,0.0
min,0.0,0.0
25%,0.0,0.0
50%,0.0,0.0
75%,0.0,0.0
max,0.0,0.0


The above analysis indicate that all null values in preciptype are dates with clear weather

In [28]:
weather['preciptype'] = weather['preciptype'].fillna('clear')

In [29]:
# windgust

According to U.S. weather observing practice, gusts are reported when the peak wind speed reaches at least 16 knots and the variation in wind speed between the peaks and lulls is at least 9 knots. The duration of a gust is usually less than 20 seconds.

Since we don't have hourly data there is no point of using windgust for our problem statement. It can also hinder with our predictions.

In [30]:
drop_columns.append('windgust')

In [31]:
weather[(weather['snow']>0) | (weather['snowdepth']>0)][['snow','snowdepth']].describe()

,snow,snowdepth
count,56.000000,56.000000
mean,2.130357,10.737500
std,3.280794,9.109646
min,0.000000,0.100000
25%,0.000000,1.700000
50%,1.000000,9.100000
75%,2.525000,18.325000
max,16.100000,31.700000


In [32]:
drop_columns

['name', 'address', 'resolvedAddress', 'severerisk', 'windgust']

In [33]:
weather = weather.drop(columns=drop_columns)

### Dataset 3: taxi-zones

In [34]:
os.listdir('dataset/taxi_zones')

['taxi_zones.dbf',
 'taxi_zones.shp.xml',
 'taxi_zones.shp',
 'taxi_zones.sbx',
 'taxi_zones.shx',
 'location_lat_long.csv',
 'taxi_zones.prj',
 'taxi_zones.sbn']

In [35]:
gdf = gpd.read_file(f"dataset/taxi_zones/taxi_zones.shp")
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 263 entries, 0 to 262
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   OBJECTID    263 non-null    int64   
 1   Shape_Leng  263 non-null    float64 
 2   Shape_Area  263 non-null    float64 
 3   zone        263 non-null    object  
 4   LocationID  263 non-null    int64   
 5   borough     263 non-null    object  
 6   geometry    263 non-null    geometry
dtypes: float64(2), geometry(1), int64(2), object(2)
memory usage: 14.5+ KB


In [36]:
gdf.head()

,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,geometry
0,1,0.116357,0.000782,Newark Airport,1,EWR,"POLYGON ((933100.918 192536.086, 933091.011 19..."
1,2,0.433470,0.004866,Jamaica Bay,2,Queens,"MULTIPOLYGON (((1033269.244 172126.008, 103343..."
2,3,0.084341,0.000314,Allerton/Pelham Gardens,3,Bronx,"POLYGON ((1026308.770 256767.698, 1026495.593 ..."
3,4,0.043567,0.000112,Alphabet City,4,Manhattan,"POLYGON ((992073.467 203714.076, 992068.667 20..."
4,5,0.092146,0.000498,Arden Heights,5,Staten Island,"POLYGON ((935843.310 144283.336, 936046.565 14..."


In [37]:
gdf['centroid'] = gdf.centroid
gdf = gdf.to_crs(epsg=4326)
gdf['centroid'] = gdf['centroid'].to_crs(epsg=4326)

In [38]:
m1 = folium.Map(location=[40.70, -73.94], zoom_start=12, tiles="Cartodb Positron")

# add centroid marker to each taxi zones
latitudes = []
longitudes = []

for _, r in gdf.iterrows():
    lat = r['centroid'].y
    latitudes.append(lat)
    lon = r['centroid'].x
    longitudes.append(lon)
    sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.0001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,style_function=lambda x: {'fillColor': 'green'})
    folium.Popup(r['zone']).add_to(geo_j)
    geo_j.add_to(m1)

    
# markers_group.add_to(m)
gdf['c_latitude'] = latitudes
gdf['c_longitude'] = longitudes
# m1.save('maps/gdf.html') # save as html
m1

In [39]:
len(latitudes)

263

In [40]:
# gdf.to_csv("dataset/taxi_zones/location_lat_long.csv")

### Dataset 4: fhvhv_tripdata_2021-01.parquet to fhvhv_tripdata_2021-12.parquet

In [41]:
spark = SparkSession.builder.appName("Taxi").getOrCreate()

24/02/08 09:01:23 WARN Utils: Your hostname, Afnanurrahim.local resolves to a loopback address: 127.0.0.1; using 192.168.29.10 instead (on interface en0)
24/02/08 09:01:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/08 09:01:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [42]:
spark

In [43]:
os.listdir('dataset')

['september.parquet',
 '.DS_Store',
 'july.parquet',
 'taxi_zones',
 'new_york_weather.csv',
 'april.parquet',
 'data_dictionary_trip_records_hvfhs.pdf',
 'nyc_2021_weather.csv',
 'october.parquet',
 'public_holidays.csv',
 'june.parquet',
 'taxi_zone_lookup.csv',
 'february.parquet',
 'weather_processed.csv',
 'december.parquet',
 'august.parquet',
 '.ipynb_checkpoints',
 'november.parquet',
 'may.parquet',
 'march.parquet',
 'working_parquet_format.pdf',
 'january.parquet']

In [44]:
class PDF(object):
    def __init__(self, pdf, size=(200,200)):
        self.pdf = pdf
        self.size = size

    def _repr_html_(self):
        return '<iframe src={0} width={1[0]} height={1[1]}></iframe>'.format(self.pdf, self.size)

    def _repr_latex_(self):
        return r'\includegraphics[width=1.0\textwidth]{{{0}}}'.format(self.pdf)

In [45]:
PDF('dataset/data_dictionary_trip_records_hvfhs.pdf',size=(800,650))

In [46]:
file_rename = {'fhvhv_tripdata_2021-03.parquet': 'march.parquet',
 'fhvhv_tripdata_2021-12.parquet': 'december.parquet',
 'fhvhv_tripdata_2021-02.parquet': 'february.parquet',
 'fhvhv_tripdata_2021-09.parquet': 'september.parquet',
 'fhvhv_tripdata_2021-10.parquet': 'october.parquet',
 'fhvhv_tripdata_2021-11.parquet': 'november.parquet',
 'fhvhv_tripdata_2021-01.parquet': 'january.parquet',
 'fhvhv_tripdata_2021-08.parquet': 'august.parquet',
 'fhvhv_tripdata_2021-04.parquet': 'april.parquet',
 'fhvhv_tripdata_2021-05.parquet': 'may.parquet',
 'fhvhv_tripdata_2021-07.parquet': 'july.parquet',
 'fhvhv_tripdata_2021-06.parquet': 'june.parquet'}

In [47]:
# for file in file_rename:
#     os.rename(f'dataset/{file}', f'dataset/{file_rename[file]}')

In [48]:
files = [f for f in os.listdir('dataset') if f.endswith('.parquet')]

In [49]:
len(files)

12

In [50]:
jan_df = spark.read.parquet('dataset/january.parquet')
feb_df = spark.read.parquet('dataset/february.parquet')
march_df = spark.read.parquet('dataset/march.parquet')
april_df = spark.read.parquet('dataset/april.parquet')
may_df = spark.read.parquet('dataset/may.parquet')
june_df = spark.read.parquet('dataset/june.parquet')
july_df = spark.read.parquet('dataset/july.parquet')
aug_df = spark.read.parquet('dataset/august.parquet')
sep_df = spark.read.parquet('dataset/september.parquet')
oct_df = spark.read.parquet('dataset/october.parquet')
nov_df = spark.read.parquet('dataset/november.parquet')
dec_df = spark.read.parquet('dataset/december.parquet')

In [51]:
month_var = [jan_df, feb_df, march_df, april_df, may_df, june_df, july_df, aug_df, sep_df, oct_df, nov_df, dec_df,]

In [52]:
for month in month_var:
    print(f"shape: ({month.count()}, {len(month.columns)})")

shape: (11908468, 24)
shape: (11613942, 24)
shape: (14227393, 24)
shape: (14111371, 24)
shape: (14719171, 24)
shape: (14961892, 24)
shape: (15027174, 24)
shape: (14499696, 24)
shape: (14886055, 24)
shape: (16545356, 24)
shape: (16041639, 24)
shape: (16054495, 24)


In [53]:
total = 0
for month in month_var:
    total+= month.count()

In [54]:
total

174596652

In [55]:
combined_df = jan_df.union(feb_df)

for month in month_var[2:]:
    combined_df = combined_df.union(month)

In [56]:
f"shape: ({combined_df.count()}, {len(combined_df.columns)})"

'shape: (174596652, 24)'

In [57]:
combined_df.dtypes

[('hvfhs_license_num', 'string'),
 ('dispatching_base_num', 'string'),
 ('originating_base_num', 'string'),
 ('request_datetime', 'timestamp_ntz'),
 ('on_scene_datetime', 'timestamp_ntz'),
 ('pickup_datetime', 'timestamp_ntz'),
 ('dropoff_datetime', 'timestamp_ntz'),
 ('PULocationID', 'bigint'),
 ('DOLocationID', 'bigint'),
 ('trip_miles', 'double'),
 ('trip_time', 'bigint'),
 ('base_passenger_fare', 'double'),
 ('tolls', 'double'),
 ('bcf', 'double'),
 ('sales_tax', 'double'),
 ('congestion_surcharge', 'double'),
 ('airport_fee', 'double'),
 ('tips', 'double'),
 ('driver_pay', 'double'),
 ('shared_request_flag', 'string'),
 ('shared_match_flag', 'string'),
 ('access_a_ride_flag', 'string'),
 ('wav_request_flag', 'string'),
 ('wav_match_flag', 'string')]

In [58]:
combined_df.groupBy('hvfhs_license_num').count().show()

+-----------------+---------+
|hvfhs_license_num|    count|
+-----------------+---------+
|           HV0004|   891819|
|           HV0005| 47575769|
|           HV0003|126129064|
+-----------------+---------+



- HV0003: Uber
- HV0004: Via
- HV0005: Lyft

In [59]:
combined_df.select('originating_base_num').distinct().count()

44

In [60]:
combined_df.select('dispatching_base_num').distinct().count()

34

In [61]:
combined_df.select('originating_base_num').distinct().show()

+--------------------+
|originating_base_num|
+--------------------+
|              B02876|
|              B00887|
|              B03136|
|              B02877|
|              B02869|
|              B02883|
|              B02835|
|              B02884|
|              B02729|
|              B02880|
|              B02878|
|              B02836|
|              B02872|
|              B02512|
|              B02867|
|              B02866|
|              B02871|
|              B02889|
|              B02826|
|              B02026|
+--------------------+
only showing top 20 rows



In [62]:
combined_df.select('dispatching_base_num').distinct().show()

+--------------------+
|dispatching_base_num|
+--------------------+
|              B02876|
|              B03136|
|              B02877|
|              B02869|
|              B02883|
|              B02835|
|              B02884|
|              B02880|
|              B02878|
|              B02836|
|              B02872|
|              B02512|
|              B02867|
|              B02866|
|              B02871|
|              B02889|
|              B02844|
|              B02510|
|              B02888|
|              B02682|
+--------------------+
only showing top 20 rows



In [63]:
combined_df.filter(combined_df.originating_base_num.isNull()).count()

48429487

In [64]:
combined_df.filter(combined_df.dispatching_base_num.isNull()).count()

0

In [65]:
combined_df.filter((combined_df.originating_base_num.isNotNull()) &
                   (combined_df.originating_base_num != combined_df.dispatching_base_num)).count()

44376

In [66]:
org_collect = combined_df.select('originating_base_num').distinct().collect()
disp_collect = combined_df.select('dispatching_base_num').distinct().collect()

In [67]:
org_base = [row.originating_base_num for row in org_collect]
disp_base = [row.dispatching_base_num for row in disp_collect]

In [68]:
[value for value in org_base if value not in disp_base]

['B00887',
 'B02729',
 'B02826',
 'B02026',
 'B00692',
 'B00457',
 'B03153',
 'B00446',
 'B02390',
 'B01985',
 None]

In [69]:
[value for value in disp_base if value not in org_base]

['B02844']

In [70]:
spark.sparkContext.stop()

### Dataset 5: public_holidays.csv

In [71]:
holidays = pd.read_csv('dataset/public_holidays.csv', parse_dates=['Date'])

In [72]:
holidays

,Date,Day,Holiday
0,2021-01-01,Friday,New Year's Day
1,2021-01-18,Monday,"Birthday of Martin Luther King, Jr."
2,2021-02-12,Friday,Lincoln's Birthday
3,2021-02-15,Monday,Washington's Birthday
4,2021-05-31,Monday,Memorial Day
5,2021-07-05,Monday,Independence Day
6,2021-09-06,Monday,Labor Day
7,2021-10-11,Monday,Columbus Day
8,2021-11-02,Tuesday,Election Day
9,2021-11-11,Thursday,Veterans Day


In [73]:
holidays.dtypes

Date       datetime64[ns]
Day                object
Holiday            object
dtype: object

### Dataset 6: weather.csv

In [74]:
weather = pd.read_csv('dataset/new_york_weather.csv')
weather.head()

,Unnamed: 0,valid,tmpf,dwpf,relh,drct,sknt,p01i,alti,mslp,...,skyl2,skyl3,skyl4,wxcodes,peak_wind_gust,peak_wind_drct,peak_wind_time,feel,metar,snowdepth
0,0,2021-01-01 00:51,40.0,26.1,57.47,320.0,13.0,0.0,30.34,1027.4,...,15000.0,NaN,NaN,NaN,NaN,NaN,NaN,31.69,KJFK 010051Z 32013KT 10SM FEW030 OVC150 04/M03...,NaN
1,1,2021-01-01 01:51,39.0,24.1,54.77,350.0,9.0,0.0,30.35,1027.8,...,17000.0,NaN,NaN,NaN,NaN,NaN,NaN,32.22,KJFK 010151Z 35009KT 10SM FEW030 SCT170 04/M04...,NaN
2,2,2021-01-01 02:51,36.0,24.1,61.62,340.0,7.0,0.0,30.38,1028.8,...,25000.0,NaN,NaN,NaN,NaN,NaN,NaN,29.65,KJFK 010251Z 34007KT 10SM FEW160 FEW250 02/M04...,NaN
3,3,2021-01-01 03:51,36.0,25.0,63.98,340.0,10.0,0.0,30.41,1029.7,...,25000.0,NaN,NaN,NaN,NaN,NaN,NaN,27.98,KJFK 010351Z 34010KT 10SM FEW160 FEW250 02/M04...,NaN
4,4,2021-01-01 04:51,35.0,24.1,63.86,350.0,9.0,0.0,30.42,1030.0,...,25000.0,NaN,NaN,NaN,NaN,NaN,NaN,27.37,KJFK 010451Z 35009KT 10SM FEW170 FEW250 02/M04...,NaN


In [75]:
weather.dtypes

Unnamed: 0          int64
valid              object
tmpf              float64
dwpf              float64
relh              float64
drct              float64
sknt              float64
p01i              float64
alti              float64
mslp              float64
vsby              float64
gust              float64
skyc1              object
skyc2              object
skyc3              object
skyc4              object
skyl1             float64
skyl2             float64
skyl3             float64
skyl4             float64
wxcodes            object
peak_wind_gust    float64
peak_wind_drct    float64
peak_wind_time     object
feel              float64
metar              object
snowdepth         float64
dtype: object

In [76]:
weather.isnull().sum()

Unnamed: 0           0
valid                0
tmpf                 0
dwpf                 0
relh                 0
drct                72
sknt                 4
p01i               776
alti                 0
mslp               950
vsby                 1
gust              8359
skyc1                1
skyc2             3606
skyc3             6831
skyc4             8976
skyl1              447
skyl2             3606
skyl3             6831
skyl4             8976
wxcodes           8104
peak_wind_gust    8704
peak_wind_drct    8704
peak_wind_time    8704
feel                 0
metar                0
snowdepth         9608
dtype: int64

In [78]:
# weather.station.value_counts()

In [ ]:
# weather = weather.drop(columns=['ice_accretion_1hr', 'ice_accretion_3hr', 'ice_accretion_6hr', 'station'])

In [ ]:
# weather.to_csv('dataset/new_york_weather.csv')